In [2958]:
import re
import numpy as np
import pandas as pd

In [2959]:
import numpy as np

df = pd.read_csv('/Users/bhaskaradhikari/nlp/Messages.csv')
df_key = pd.read_csv('/Users/bhaskaradhikari/nlp/Keywords.csv')

### Preprocess sentences in the corpus

In [2960]:
# Preprocess sentences in the corpus

corpus = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower().split()
    review = ' '.join(review)
    corpus.append(review)


In [2961]:
corpus

['desk',
 'flat pack desk cupboard pedestal drawers',
 'door step delivery',
 'ground floor',
 'no number i m afraid i can wait here on the web',
 'day bed',
 'yes it is the frame for a single bed and mattresses',
 'ground floor to ground floor it will be dismantled',
 'just the bed frame which will be dismantled and the mattress',
 'specific time',
 'yes please',
 'ok i ll wait for your confirmation thanks',
 'business move large printers requiring four people to lift shelving storage units',
 'first floor no lift but wide staircase',
 'first floor no lift',
 'men',
 'can you quote for two people then',
 'can i just confirm what that includes please',
 'ok sorry i don t think i was clear it s a business move so they are the main items but there would be more than that the items are quite large and heavy i also forgot to mention one of the printers is round the corner in my house l uh so would also need to add a pick up from there',
 'there s a paper drill mainly boxes storage units de

In [2962]:
# Preprocess keywords
corpus_key = []
for i in range(len(df_key)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df_key['keyword'][i])
    review = review.lower().split()
    review = ' '.join(review)
    corpus_key.append(review)

In [2963]:
corpus_key

['basement',
 'bungalow',
 'delivery',
 'downstair',
 'front door',
 'garage',
 'garden',
 'home',
 'level',
 'pavement',
 'pickup',
 'right now',
 'stairs',
 'storage',
 'street level']

### find sentences containg words present in keyword.csv

In [2964]:
def find_sentences_with_similar_words(corpus, similar_words):
    """
    Search for sentences in the corpus that contain any of the similar words.

    Parameters:
    - corpus: List[str], the corpus of text (each element is a sentence or document).
    - similar_words: List[str], words similar to the search word.

    Returns:
    - Dict[str, List[str]], a dictionary where keys are similar words and values are lists of sentences containing those words.
    """
    # Initialize a dictionary to hold the results
    results = {word: [] for word in similar_words}
    
    # Iterate over the corpus to find sentences containing any of the similar words
    for sentence in corpus:
        for word in similar_words:
            if word in sentence.lower():  # Case-insensitive search
                results[word].append(sentence)
        
    counts = {word: len(sentences) for word, sentences in results.items()}
    
    data_for_df = [(word, "; ".join(sentences)) for word, sentences in results.items()]
 
    return results, counts , data_for_df


# Assuming 'model' is your trained Word2Vec model
#similar_words = [word for word, _ in model.wv.similar_by_word('delivery')]
similar_words=corpus_key
# Assuming 'corpus' is a list of sentences

# Find sentences containing similar words
sentences_with_similar_words , sentence_counts , data_for_df = find_sentences_with_similar_words(corpus, similar_words)

# Print the results
for word, sentences in sentences_with_similar_words.items():
    print(f"Sentences containing '{word}':")
    for sentence in sentences:
        print(f"- {sentence}")
    print()  # Blank line for readability

df_counts = pd.DataFrame(list(sentence_counts.items()), columns=['Word', 'Number of Sentences'])
df_sentences = pd.DataFrame(data_for_df, columns=['Word', 'Sentences'])



Sentences containing 'basement':
- no lift pick up first floor flat delivery is basement flat
- basement flat to basement flat no lift
- basement flat no lift so they have to take stairs one floor
- basement no stairs to climb
- destination is ground floor apartment moving from duplex ground floor and basement bedroom items from basement stairs
- other end is a basement
- basement no lift around steepish narrow stairs

Sentences containing 'bungalow':
- it s a bungalow
- going to a bungalow
- my place gf i dont know about the lady with the carpet bungalow
- a bungalow
- bungalow property
- ground floor flat to bungalow and back
- ground floor flat to bungalow
- on ground floor one into a garage the other items into a ground floor bungalow and collect sofa from a house that has a few steps going up to it
- it s in first floor flat going in to a bungalow
- it s a bungalow
- we live in a bungalow so no stairs my daughters garage is on the flat outside her house
- you don t need a lift as 

### count no of sentences

In [2965]:
df_counts_sorted = df_counts.sort_values(by='Number of Sentences', ascending=False)
df_counts_sorted

,Word,Number of Sentences
12,stairs,192
2,delivery,102
6,garden,51
13,storage,50
5,garage,41
3,downstair,31
1,bungalow,30
4,front door,30
8,level,25
7,home,22


### Function to count occurrences of each keyword or phrase in the corpus


In [2966]:
# Function to count occurrences of each keyword or phrase in the corpus
def count_keyword_occurrences(corpus, keywords):
    # Convert the entire corpus into a single string for easier searching
    corpus_text = ' '.join(corpus).lower()
    
    # Initialize a dictionary to hold the count of occurrences for each keyword
    keyword_occurrences = {}
    
    # Loop through each keyword to find and count its occurrences in the corpus text
    for keyword in keywords:
        # Normalize keyword to ensure consistent matching
        normalized_keyword = keyword.lower()
        # Count occurrences of the keyword in the corpus text
        count = corpus_text.count(normalized_keyword)
        # Store the count in the dictionary
        keyword_occurrences[keyword] = count
    
    return keyword_occurrences

# Assuming corpus and corpus_key are defined as before
# Normalize keywords since we're dealing with exact matches
keywords = [keyword.lower() for keyword in corpus_key]

# Count occurrences of each keyword or phrase in the corpus
keyword_occurrences = count_keyword_occurrences(corpus, keywords)

# keyword_occurrences now contains the count of each keyword or phrase present in the corpus


### keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2967]:
# Assuming keyword_occurrences is the dictionary with keyword counts
keyword_occurrences_sorted = sorted(keyword_occurrences.items(), key=lambda x: x[1], reverse=True)

# Convert sorted list of tuples back to a dictionary if you prefer dictionary format
keyword_occurrences_sorted_dict = dict(keyword_occurrences_sorted)

# keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2968]:
keyword_occurrences_sorted_dict

{'stairs': 215,
 'delivery': 104,
 'garden': 58,
 'storage': 51,
 'garage': 42,
 'downstair': 33,
 'bungalow': 30,
 'front door': 30,
 'level': 30,
 'home': 23,
 'pickup': 17,
 'basement': 9,
 'street level': 8,
 'pavement': 3,
 'right now': 3}